In [52]:
import sympy as sp
from typing import List
from sympy import sin, cos, pi, sqrt, acos, simplify, atan
import math
sp.init_printing()


def homogeneous(rotation: sp.Matrix = sp.eye(3), translation: sp.Matrix = sp.zeros(3, 1)) -> sp.Matrix:
    return rotation.row_join(translation).col_join(sp.Matrix([[0, 0, 0, 1]]))

def dh(rotation, twist, displacement, offset):
    rotation_mat = sp.Matrix([
        [cos(rotation), -sin(rotation)*cos(twist),  sin(rotation)*sin(twist)],
        [sin(rotation), cos(rotation)*cos(twist),   -cos(rotation)*sin(twist)],
        [0,             sin(twist),                 cos(twist)],
    ])
    translation = sp.Matrix([
        [offset*cos(rotation)],
        [offset*sin(rotation)],
        [displacement],
    ])
    return rotation_mat, translation

def rotation(homogeneous: sp.Matrix):
    return homogeneous[:3, :3]

def translation(homogeneous: sp.Matrix):
    return homogeneous[:3, 3:]

def jacobian(transforms: List[sp.Matrix], joint_types: List[sp.Matrix], base_z: sp.Matrix = sp.Matrix([0, 0, 1])):
    transforms_chained = [homogeneous()]
    z_unit_vecs = []
    # chain the transforms
    for i, transform in enumerate(transforms):
        transforms_chained.append(transforms_chained[-1] * transform)
    # apply rotation of chained transforms to base z
    for chained_transform in transforms_chained:
        z_unit_vecs.append(rotation(chained_transform) * base_z)
    
    assert len(transforms_chained) == len(z_unit_vecs)

    jacobian = sp.zeros(6, len(transforms))
    for i, (transform, joint_type) in enumerate(zip(transforms, joint_types)):
        if joint_type == 'revolute':
            jacobian[:3, i] = z_unit_vecs[i].cross(translation(transforms_chained[-1]) - translation(transforms_chained[i]))
            jacobian[3:, i] = z_unit_vecs[i]
        elif joint_type == 'prismatic':
            jacobian[:3, i] = z_unit_vecs[i]
            jacobian[3:, i] = sp.Matrix([[0], [0], [0]])

        # angular velocity

    return jacobian

In [53]:
q_1, q_2, q_3 = sp.symbols('q_1 q_2 q_3')

joint_1 = homogeneous(*dh(q_1, -pi/2, .4, 0))
joint_2 = homogeneous(*dh(q_2, pi/2, 0, 0))
joint_3 = homogeneous(*dh(q_3, 0, .15, .1))

In [54]:
# Question 1a
z_0_0 = sp.Matrix([0, 0, 1])
display(rotation(joint_1)*z_0_0)
display(rotation(joint_1)*rotation(joint_2)*z_0_0)
J = simplify(jacobian([joint_1, joint_2, joint_3], ['revolute', 'revolute', 'revolute']))
J

⎡-sin(q₁)⎤
⎢        ⎥
⎢cos(q₁) ⎥
⎢        ⎥
⎣   0    ⎦

⎡sin(q₂)⋅cos(q₁)⎤
⎢               ⎥
⎢sin(q₁)⋅sin(q₂)⎥
⎢               ⎥
⎣    cos(q₂)    ⎦

⎡-0.15⋅sin(q₁)⋅sin(q₂) - 0.1⋅sin(q₁)⋅cos(q₂)⋅cos(q₃) - 0.1⋅sin(q₃)⋅cos(q₁)  (-
⎢                                                                             
⎢-0.1⋅sin(q₁)⋅sin(q₃) + 0.15⋅sin(q₂)⋅cos(q₁) + 0.1⋅cos(q₁)⋅cos(q₂)⋅cos(q₃)  (-
⎢                                                                             
⎢                                    0                                        
⎢                                                                             
⎢                                    0                                        
⎢                                                                             
⎢                                    0                                        
⎢                                                                             
⎣                                    1                                        

0.1⋅sin(q₂)⋅cos(q₃) + 0.15⋅cos(q₂))⋅cos(q₁)  -0.1⋅sin(q₁)⋅cos(q₃) - 0.1⋅sin(q₃
                                                   

In [55]:
# Question 1b
f_x, f_y, f_z, g_x, g_y, g_z = sp.symbols('f_x f_y f_z g_x g_y g_z')
F = sp.Matrix([f_x, f_y, f_z, g_x, g_y, g_z])
t = simplify(J.T*F)
display(t)
o = sp.Matrix([0, 0, 0, 1])
p = -(joint_1*joint_2*joint_3*o)
R_0_e = rotation(joint_1)*rotation(joint_2)*rotation(joint_3)
def skew(v: sp.Matrix):
    return sp.Matrix([
        [0, -v[2], v[1]],
        [v[2], 0, -v[0]],
        [-v[1], v[0], 0],
    ])
S_0_e0 = sp.zeros(6, 6)
S_0_e0[:3, :3] = R_0_e.T
S_0_e0[:3, 3:] = skew(p)*R_0_e.T
S_0_e0[3:, 3:] = R_0_e.T
G = sp.Matrix([0, 0, 0, .04, 0, 0])
f = F.evalf(subs={f_x: -10, f_y: 0, f_z: -8, g_x: 0, g_y: 0, g_z: 0})+(S_0_e0*G).evalf(subs={q_1: math.radians(90), q_2: math.radians(45), q_3: 0})

(J.T*(F+(S_0_e0*G))).evalf(subs={f_x: -10, f_y: 0, f_z: -8, g_x: 0, g_y: 0, g_z: 0,q_1: math.radians(90), q_2: math.radians(45), q_3: 0})

⎡           -fₓ⋅(0.15⋅sin(q₁)⋅sin(q₂) + 0.1⋅sin(q₁)⋅cos(q₂)⋅cos(q₃) + 0.1⋅sin(
⎢                                                                             
⎢         -fₓ⋅(0.1⋅sin(q₂)⋅cos(q₃) - 0.15⋅cos(q₂))⋅cos(q₁) - f_y⋅(0.1⋅sin(q₂)⋅
⎢                                                                             
⎣-0.1⋅fₓ⋅(sin(q₁)⋅cos(q₃) + sin(q₃)⋅cos(q₁)⋅cos(q₂)) - 0.1⋅f_y⋅(sin(q₁)⋅sin(q₃

q₃)⋅cos(q₁)) + f_y⋅(-0.1⋅sin(q₁)⋅sin(q₃) + 0.15⋅sin(q₂)⋅cos(q₁) + 0.1⋅cos(q₁)⋅
                                                                              
cos(q₃) - 0.15⋅cos(q₂))⋅sin(q₁) - f_z⋅(0.15⋅sin(q₂) + 0.1⋅cos(q₂)⋅cos(q₃)) - g
                                                                              
)⋅cos(q₂) - cos(q₁)⋅cos(q₃)) + 0.1⋅f_z⋅sin(q₂)⋅sin(q₃) + gₓ⋅sin(q₂)⋅cos(q₁) + 

cos(q₂)⋅cos(q₃)) + g_z           ⎤
                                 ⎥
ₓ⋅sin(q₁) + g_y⋅cos(q₁)          ⎥
                                 ⎥
g_y⋅sin(q₁)⋅sin(q₂) + g_z⋅cos(q₂)⎦

⎡1.77084538009111 ⎤
⎢                 ⎥
⎢1.41421356237309 ⎥
⎢                 ⎥
⎣0.973457150108775⎦